In [ ]:
import os
import pandas as pd
from datetime import datetime

# ------------------------
# Constants for File Paths
# ------------------------
RESPONSES_FILE = 'responses.csv'
LOG_FILE = 'chat_logs.csv'

# ------------------------
# Load and Save Responses
# ------------------------
def load_responses():
    if not os.path.exists(RESPONSES_FILE):
        # Default dataset if the file doesn't exist
        default_responses = pd.DataFrame({
            "Category": ["Product Information", "Order Status", "Returns and Refunds", "Technical Support", "General FAQs"],
            "Sample Query": [
                "What colors are available?",
                "Where is my order?",
                "How to return an item?",
                "My device isn't working",
                "What are your hours?"
            ],
            "Response": [
                "We have Red, Blue, and Green available.",
                "Track your order here: [link].",
                "You can initiate a return via our portal.",
                "Please follow our troubleshooting guide here: [link].",
                "We are open from 9 AM to 9 PM."
            ]
        })
        default_responses.to_csv(RESPONSES_FILE, index=False)
        return default_responses
    return pd.read_csv(RESPONSES_FILE)

def save_responses(data):
    data.to_csv(RESPONSES_FILE, index=False)

# ------------------------
# Chatbot Core Functions
# ------------------------
def find_response(query, responses):
    query = query.lower()
    for _, row in responses.iterrows():
        if row["Sample Query"].lower() in query:
            return row["Category"], row["Response"]
    return "Uncategorized", "Sorry, I couldn't understand that. Please try again."

def log_interaction(timestamp, query, category, response, sentiment="Neutral"):
    log_entry = {
        "Timestamp": timestamp,
        "Query": query,
        "Category": category,
        "Response": response,
        "Sentiment": sentiment
    }
    if not os.path.exists(LOG_FILE):
        pd.DataFrame([log_entry]).to_csv(LOG_FILE, index=False)
    else:
        pd.DataFrame([log_entry]).to_csv(LOG_FILE, mode='a', header=False, index=False)

def analyze_sentiment(query):
    negative_words = ["not", "don't", "can't", "won't", "bad", "issue", "problem"]
    positive_words = ["great", "good", "excellent", "amazing", "happy", "satisfied"]
    if any(word in query.lower() for word in negative_words):
        return "Negative"
    elif any(word in query.lower() for word in positive_words):
        return "Positive"
    return "Neutral"

# ------------------------
# Admin Functions
# ------------------------
def update_responses():
    responses = load_responses()
    print("\n--- Update Responses ---")
    category = input("Enter the category to update or add: ")
    query = input("Enter a new sample query: ")
    response = input("Enter the corresponding response: ")
    new_entry = {"Category": category, "Sample Query": query, "Response": response}
    responses = responses.append(new_entry, ignore_index=True)
    save_responses(responses)
    print(f"Updated category '{category}' successfully!")

# ------------------------
# View Logs
# ------------------------
def view_logs():
    if not os.path.exists(LOG_FILE):
        print("No logs available.")
        return
    logs = pd.read_csv(LOG_FILE)
    print(logs)
    filter_choice = input("Do you want to filter logs? (yes/no): ").lower()
    if filter_choice == "yes":
        print("1. Filter by Category")
        print("2. Filter by Date")
        choice = input("Enter your choice: ")
        if choice == "1":
            category = input("Enter the category to filter by: ")
            filtered_logs = logs[logs['Category'] == category]
        elif choice == "2":
            start_date = input("Enter start date (YYYY-MM-DD): ")
            end_date = input("Enter end date (YYYY-MM-DD): ")
            filtered_logs = logs[(logs['Timestamp'] >= start_date) & (logs['Timestamp'] <= end_date)]
        else:
            print("Invalid choice. Showing all logs.")
            filtered_logs = logs
        print(filtered_logs)

# ------------------------
# Main Chatbot Function
# ------------------------
def start_chat():
    responses = load_responses()
    print("\n--- Chatbot Simulation ---")
    print("Type 'exit' to end the session.")
    while True:
        user_query = input("You: ")
        if user_query.lower() == "exit":
            print("Chatbot: Thank you! Goodbye.")
            break
        sentiment = analyze_sentiment(user_query)
        category, response = find_response(user_query, responses)
        log_interaction(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), user_query, category, response, sentiment)
        print(f"Chatbot: {response}")

# ------------------------
# Main Menu
# ------------------------
def main_menu():
    while True:
        print("\n--- Chatbot System ---")
        print("1. Start Chat")
        print("2. View Logs")
        print("3. Update Responses (Admin)")
        print("4. Exit")
        choice = input("Enter your choice: ")
        if choice == "1":
            start_chat()
        elif choice == "2":
            view_logs()
        elif choice == "3":
            update_responses()
        elif choice == "4":
            print("Exiting the system. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main_menu()



--- Chatbot System ---
1. Start Chat
2. View Logs
3. Update Responses (Admin)
4. Exit
Enter your choice: 1

--- Chatbot Simulation ---
Type 'exit' to end the session.
You: What are your hours?
Chatbot: We are open from 9 AM to 9 PM.
You: exit
Chatbot: Thank you! Goodbye.

--- Chatbot System ---
1. Start Chat
2. View Logs
3. Update Responses (Admin)
4. Exit
Enter your choice: 2
             Timestamp                 Query      Category  \
0  2024-12-05 10:44:47  What are your hours?  General FAQs   

                         Response Sentiment  
0  We are open from 9 AM to 9 PM.   Neutral  
Do you want to filter logs? (yes/no): exit

--- Chatbot System ---
1. Start Chat
2. View Logs
3. Update Responses (Admin)
4. Exit
Enter your choice: 4
Exiting the system. Goodbye!
